In [2]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src import api
app = api.Trainer.initialize(stage = 0 , resume = 0 , checkname= 1)
app.go()

24-06-28 22:56:03|MOD:display     |: Model Specifics:
24-06-28 22:56:03|MOD:display     |: Start Process [Data] at Fri Jun 28 22:56:03 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to ple_gru_day_ShortTest!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
Callback : DetailedAlphaAnalysis() , record and concat each model to Alpha model instance
{'random_seed': None,
 'model_name': 'ple_gru_day_Short

24-06-28 22:56:05|MOD:display     |: Finish Process [Data], Cost 1.4 Secs
24-06-28 22:56:05|MOD:display     |: Start Process [Fit] at Fri Jun 28 22:56:05 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/43 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : 0.02115: 100%|██████████| 11/11 [00:11<00:00,  1.07s/it] 
FirstBite Ep#  0 : loss  3.96300, train 0.01583, valid-0.03244, best-0.0324, lr1.3e-03
Valid Ep#  1 score : -0.02199: 100%|██████████| 11/11 [00:12<00:00,  1.16s/it]
FirstBite Ep#  1 : loss  3.76727, train 0.04536, valid 0.02207, best 0.0221, lr2.5e-03
Valid Ep#  2 score : -0.03724: 100%|██████████| 11/11 [00:10<00:00,  1.00it/s]
FirstBite Ep#  2 : loss  3.50294, train 0.05701, valid 0.02881, best 0.0288, lr3.8e-03
Valid Ep#  3 score : -0.06030: 100%|██████████| 11/11 [00:11<00:00,  1.03s/it]
FirstBite Ep#  3 : loss  3.28882, train 0.06901, valid 0.03558, best 0.0356, lr5.0e-03
24-06-28 23:05:15|MOD:display     |: ple_gru_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0690 Valid 0.0356 BestVal 0.0356|Cost  9.1Min,109.6Sec/Ep
24-06-28 23:05:15|MOD:display     |: Finish Process [Fit], Cost 0.2 Hours, 9.2 Min/model, 137.5 Sec/Epoch
24-06-28 23:05:15|MOD:display     |: Start Process [Test] at Fri 

Group optimization of 3 alphas , 3 benchmarks , 2 lags , 7 dates , (126 opts) start!
Not accurate but assessed as success!
Accuarcy(Is Accurate=[False] ,,
          lin_ub_bias=(X)-1.200461106964923e-06,
          lin_lb_bias=(√)1.2903060992060233e-08,
          bnd_ub_bias=(√)-4.576457733607153e-09,
          bnd_lb_bias=(√)0.0,
          excess_turn=(X)-1.0788637458958306e-06
Group optimization Finished , Total time: 22.28 secs, Setup time: 3.44 secs, Calc time: 18.84 secs, Each optim time: 0.15
0@best.csi300 accounting start... Total time: 0.68 secs, Each period time: 0.10 secs
0@best.csi500 accounting start... Total time: 0.51 secs, Each period time: 0.07 secs
0@best.csi1000 accounting start... Total time: 0.58 secs, Each period time: 0.08 secs
0@best.csi300.1 accounting start... Total time: 0.24 secs, Each period time: 0.03 secs
0@best.csi500.1 accounting start... Total time: 0.18 secs, Each period time: 0.03 secs
0@best.csi1000.1 accounting start... Total time: 0.24 secs, Each pe

24-06-28 23:09:50|MOD:time        |: Main Process Finished! Cost 13 Minutes 46.7 Seconds


Analytic datas are saved to d:\Coding\learndl\learndl\model/ple_gru_day_ShortTest/analysis_data.xlsx
Analytic plots are saved to d:\Coding\learndl\learndl\model/ple_gru_day_ShortTest/analysis_plot.pdf
              hook_name  num_calls  total_time   avg_time
10       on_train_batch        172  416.200322   2.419769
27        on_test_batch        192  208.763996   1.087312
21     on_fit_model_end          1   84.688895  84.688895
15  on_validation_batch         44   46.760696   1.062743
4    on_fit_model_start          1    1.873470   1.873470


In [ ]:
from src import api as API
API.DataAPI.reconstruct_train_data()
API.Trainer.update_models()

predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 22.28 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 21.73 secs
 --> labels blocks merging (2)...... finished! Cost 4.75 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 114.89 secs
y blocks loading finished! Cost 269.22 secs
y blocks process...... finished! Cost 128.26 secs
y blocks masking...... finished! Cost 1.38 secs
y blocks saving ...... finished! Cost 2.88 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 402.61 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 33.11 secs
day blocks loading finished! Cost 33.57 secs
day blocks process...... finished! Cost 5.12 secs
day blocks masking...... finished! Cost 1.16 secs


In [ ]:
from src import api as API
API.Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090


24-06-24 01:10:43|MOD:display     |: Model Specifics:
24-06-24 01:10:43|MOD:display     |: Start Process [Data] at Mon Jun 24 01:10:43 2024!


--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast', '

24-06-24 01:10:51|MOD:display     |: Finish Process [Data], Cost 7.9 Secs
24-06-24 01:10:51|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:10:51 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


...

FirstBite Ep#  0 : loss  1.00382, train-0.00516, valid-0.03649, best-0.0365, lr1.3e-03
FirstBite Ep#  5 : loss  0.86311, train 0.13764, valid 0.09228, best 0.0923, lr2.5e-03
FirstBite Ep# 10 : loss  0.84547, train 0.15544, valid 0.09630, best 0.0963, lr1.9e-03
FirstBite Ep# 15 : loss  0.83038, train 0.17066, valid 0.08826, best 0.0963, lr1.0e-07
FirstBite Ep# 20 : loss  0.82732, train 0.17363, valid 0.08575, best 0.0963, lr9.4e-04
FirstBite Ep# 25 : loss  0.81729, train 0.18370, valid 0.08407, best 0.0963, lr3.1e-04
FirstBite Ep# 30 : loss  0.81421, train 0.18647, valid 0.08262, best 0.0963, lr1.6e-04
24-06-24 01:27:46|MOD:display     |: gruRTN_day #1 @20231201|FirstBite Ep# 31 EarlyStop|Train 0.1865 Valid 0.0826 BestVal 0.0826|Cost  1.9Min,  3.6Sec/Ep
FirstBite Ep#  0 : loss  1.00293, train-0.00341, valid-0.00344, best-0.0034, lr1.3e-03
FirstBite Ep#  5 : loss  0.87709, train 0.12551, valid 0.08796, best 0.1129, lr2.5e-03
FirstBite Ep# 10 : loss  0.86180, train 0.13731, valid 0.10436,

--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gruRES_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast'

24-06-24 01:36:34|MOD:display     |: Finish Process [Data], Cost 2.0 Secs
24-06-24 01:36:34|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:36:34 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


FirstBite Ep#  0 : loss  1.01252, train-0.01521, valid-0.03742, best-0.0374, lr1.3e-03
FirstBite Ep#  5 : loss  0.91026, train 0.08135, valid 0.03185, best 0.0599, lr2.5e-03
FirstBite Ep# 10 : loss  0.89291, train 0.09436, valid 0.04522, best 0.0599, lr1.9e-03
FirstBite Ep# 15 : loss  0.87755, train 0.10648, valid 0.03959, best 0.0599, lr1.0e-07
FirstBite Ep# 20 : loss  0.87426, train 0.10831, valid 0.03543, best 0.0599, lr9.4e-04
FirstBite Ep# 21 : loss  0.87082, train 0.11088, valid 0.03980, best 0.0599, lr6.3e-04, Next attempt goes!
Retrain#1 Ep#  0 : loss  0.99637, train 0.00462, valid 0.03106, best 0.0311, lr1.3e-04
Retrain#1 Ep#  5 : loss  0.94196, train 0.05698, valid 0.03578, best 0.0560, lr2.5e-04
Retrain#1 Ep# 10 : loss  0.92553, train 0.06750, valid 0.03080, best 0.0560, lr1.9e-04
Retrain#1 Ep# 15 : loss  0.91653, train 0.07499, valid 0.03537, best 0.0560, lr1.0e-07
Retrain#1 Ep# 20 : loss  0.91509, train 0.07561, valid 0.04185, best 0.0560, lr9.4e-05
Retrain#1 Ep# 21 : loss